In [1]:
#------ manually fix import for now
import sys
sys.path.append('../src/')
#------

import numpy as np
import glob
from redcross.datacube import Datacube
# from redcross.read import read_harpsn


%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'datacube'

## Read dataset of `e2ds` files (HARPS-N)
The function reads a set of `M` frames with `N` orders each, with flux values for each wavelength channel `C`
and returns a reshaped `datacube` with shape `(N,M,C)`

In [ ]:
data_dir = '/home/dario/AstronomyLeiden/MRP/wasp189/data/e2ds/night1'
files = glob.glob(data_dir+'*.fits') # select files

dc = read_harpsn(files)

In [ ]:
dc = Datacube()